# Clustering of horror movies

Loading data previously cleaned in R

In [1]:
import pandas as pd
import numpy as np

movies = pd.read_csv("horror_movies_cleaned.csv")
movies

,title,story,director,cast,Action,Adventure,Comedy,Crime,Drama,Fantasy,Mystery,Romance,Sci-Fi,Thriller
0,Gut,Family man Tom has seen something he cant forg...,Elias,"Jason Vail, Nicholas Wilder, Sarah Schoofs, Ki...",0,0,0,0,1,0,0,0,0,1
1,Treasure Chest of Horrors II,"Mona Screamalot, along with her crazy family, ...","M Kelley, Shawn C Phillips, Alex Powers","Veronica Ricci, Nicholas Adam Clark, James Cul...",0,0,1,0,0,0,0,0,0,1
2,Infidus,"In the suburbs of Rome, two disparate lives wi...",Giulio De Santi,"Bonini Mino, Massimo Caratelli, Maurizio Zaffi...",0,0,0,1,1,0,0,0,0,0
3,In Extremis,Its the end of the world at least their world.,Steve Stone,"David OHara, Isabelle Allen, Neil Pearson, Toy...",0,0,0,0,0,0,0,0,0,1
4,Ghostlight,Andrew wins a radio contest to spend one night...,Jeff Ferrell,"Eden Campbell, Lisa Coronado, David A Crellin,...",0,0,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3163,Victor Frankenstein,"Told from Igors perspective, we see the troubl...",Paul McGuigan,"Daniel Radcliffe, James McAvoy, Jessica Brown ...",0,0,0,0,1,0,0,0,1,1
3164,The Exorcism of Molly Hartley,Taking place years after The Haunting of Molly...,Steven R Monroe,"Sarah Lind, Devon Sawa, Gina Holden, Peter Mac...",0,0,0,0,0,0,0,0,0,0
3165,Talon Falls,Four teenagers on a road trip decide to take a...,Joshua Shreve,"Brad Bell, Jordyn Rudolph, Ryan Rudolph, Morga...",0,0,0,0,0,0,0,0,0,0
3166,BloodRayne: The Third Reich,Rayne fights against the Nazis in Europe durin...,Uwe Boll,"Natassia Malthe, Brendan Fletcher, Michael Par...",1,1,0,0,0,1,0,0,0,0


## Tokenization of plots

Stoppord, tokens, lowercasing, möjligen lemmatisering.

In [2]:
# Using things from lab 1
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer

nlp = spacy.load("en_core_web_sm")
stop_words = ["the", "a", "in", "but", "when", "s", "be", "can"]

def preprocess(text):
    doc = nlp(text)
    
    ret_list = [token.lemma_.lower() for token  in doc if not token.is_stop and token.lemma_.lower() not in stop_words]
    return list(filter(lambda x: x.isalpha(), ret_list))

movies["token_plot"] = movies.apply(lambda row: preprocess(row["story"]), axis = 1)
movies["token_title"] = movies.apply(lambda row: preprocess(row["title"]), axis = 1)
movies["token_title_plot"] = movies.apply(lambda row: row["token_title"] + row["token_plot"], axis = 1)

In [109]:
tid = 3011
print(movies.token_plot[tid])
print(movies.token_title[tid])
print(movies.token_title_plot[tid])
print(movies.story[tid])

['resident', 'coastal', 'town', 'learn', 'deathly', 'consequence', 'secret', 'share', 'mysterious', 'woman', 'seek', 'shelter', 'local', 'resort']
['byzantium']
['byzantium', 'resident', 'coastal', 'town', 'learn', 'deathly', 'consequence', 'secret', 'share', 'mysterious', 'woman', 'seek', 'shelter', 'local', 'resort']
Residents of a coastal town learn, with deathly consequences, the secret shared by the two mysterious women who have sought shelter at a local resort.


In [3]:
# Funkar inte just nu, måste ha både splittad lista och mening lagrad nånstans...
vectorizer_p = TfidfVectorizer(stop_words = 'english')
plots_tfidf = vectorizer_p.fit_transform(movies.apply(lambda row: " ".join(row["token_plot"]), axis=1))
vectorizer_tp = TfidfVectorizer(stop_words = 'english')
title_plots_tfidf = vectorizer_tp.fit_transform(movies.apply(lambda row: " ".join(row["token_title_plot"]), axis=1))

## K-means-clustering

### Tuning number of clusters

In [4]:
from sklearn.cluster import KMeans

def inertia(model, data):
    dists = model.transform(data)
    min_dist = np.amin(dists, axis = 1)
    return sum(np.square(min_dist))


max_clus = 13

plot_models = [KMeans(n_clusters=nclus, n_init=6, n_jobs=-1, random_state = max_clus).fit(plots_tfidf) for nclus in range (6,max_clus)]
inertias = [inertia(model, plots_tfidf) for model in plot_models]

In [5]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.scatter(range(6,max_clus), inertias)
plt.plot(range(6,max_clus), inertias)
plt.show()

ModuleNotFoundError: No module named 'matplotlib'

Elbow som i labben...

## Optics-clustering

In [ ]:
from sklearn.cluster import DBSCAN

#optic_plots =  OPTICS(n_jobs=-1,  metric='cosine').fit(plots_tfidf.toarray())
dbs_plot = DBSCAN(n_jobs=-1,  eps=0.00000005, metric = "cosine").fit(plots_tfidf.toarray())

In [ ]:
pd.Series(dbs_plot.labels_).value_counts()
plots_tfidf[0:3,0:3]

### Tuning eps and min.pts

Ska väl gå på nåt jäkla vis...

## Topic modelling

In [ ]:
from gensim.corpora import Dictionary
from gensim.models import LdaModel

# Create a dictionary representation of the documents.
dictionary = Dictionary(movies["token_title_plot"])
dictionary.filter_extremes(no_below=5, no_above=0.2)
corpus_title = [dictionary.doc2bow(d) for d in movies["token_title_plot"]]
corpus_plot = [dictionary.doc2bow(d) for d in movies["token_plot"]]

In [ ]:
print(dictionary)

In [ ]:
ntopics = 10
nwords=15
#id2word = dictionary.id2token #suddenly returns empty and kills LAD... love python!
model_title = LdaModel(corpus_title, num_topics=ntopics, random_state=831117)
model_plot = LdaModel(corpus_plot, num_topics=ntopics, random_state=831117)

In [ ]:
def topic_words(model):
    topics = model.show_topics(num_words=nwords, formatted=False,log=False)
    # Lookup directly from dictionary instead of id2token
    topic_words = pd.DataFrame([[dictionary[ int(topics[i][1][j][0]) ] for j in range(nwords) ] for i in range(ntopics)])
    return topic_words
    
words_title  = topic_words(model_title)
words_title


In [ ]:
words_plot = topic_words(model_plot)
words_plot

### Evaluation

Dot-product av klustertillhörogheter?! Borde ge höga värden om de har lika klusterprobs, men kanske blir problem om man har nonsenskluster.
mappa topics till kategorier och jämföra med sanningen?

Även kolla coherence, jmf förekomster av topicord i dokumenten.